In [ ]:
# scp -r infordio-ai@192.168.68.211:/home/infordio-ai/quan/code/information_extraction/PaddleOCR/output/ser/ser_vi_layoutxlm_key_value_20231216_no1_1_no3_1_dataset1 /home/chuongphung/projects/chatgpt/XGBoost/dataset/ser_result
# scp -r infordio-ai@192.168.68.211:/home/infordio-ai/quan/code/information_extraction/PaddleOCR/output/ser/ser_vi_layoutxlm_key_value_20231216_no1_1_no3_1_dataset2 /home/chuongphung/projects/chatgpt/XGBoost/dataset/ser_result


In [1]:
import os
import json
import pickle
from sklearn.metrics import confusion_matrix


In [2]:
root_txt_ser_data1 = "/home/chuongphung/projects/chatgpt/XGBoost/dataset/ser_result/ser_vi_layoutxlm_key_value_20231216_no1_1_no3_1_dataset1/infer_results.txt"
root_txt_ser_data2 = "/home/chuongphung/projects/chatgpt/XGBoost/dataset/ser_result/ser_vi_layoutxlm_key_value_20231216_no1_1_no3_1_dataset2/infer_results.txt"



In [3]:
root_re_model = "20231122_lightgbm"

In [4]:
with open(f"{root_re_model}/clf.pkl", 'rb') as f_model:
    model_lightgbm = pickle.load(f_model)

# Prepare data

In [5]:
root_json_data1 = os.path.join(os.path.dirname(root_txt_ser_data1), "json_re")
root_json_data2 = os.path.join(os.path.dirname(root_txt_ser_data2), "json_re")

os.makedirs(root_json_data1, exist_ok=True)
os.makedirs(root_json_data2, exist_ok=True)



In [6]:
def convert(root_txt_ser_data, root_json_data):
    with open(root_txt_ser_data, "r") as f:
        contents = f.read().splitlines()

    for content in contents:
        # print(content)
        name, content_ = content.split("\t")
        name = name.split("/")[-1].replace("jpg","json")
        # print(name)
        # print(content_)
        root_out = os.path.join(root_json_data, name)
        content_ = eval(content_)["ocr_info"]
        with open(root_out,"w", encoding='utf-8') as f:
            json.dump(content_, f, ensure_ascii=False)

In [7]:
convert(root_txt_ser_data1, root_json_data1)
convert(root_txt_ser_data2, root_json_data2)


# Calculate metrics

In [8]:
# Importing Required Library
import pandas as pd
# import lightgbm as lgb
 
# Similarly LGBMRegressor can also be imported for a regression model.
# from lightgbm import LGBMClassifier

import os
import pandas as pd
import cv2
from box_utils.boxes import *
# from data.visualization import Visualization

# import glob
import pickle
# import itertools
from tqdm import tqdm
from modules.kv_embedding_full_features import KVEmbedding
import numpy as np
# import time
from sklearn.metrics import f1_score, precision_score, recall_score
import json
import warnings
warnings.filterwarnings("ignore")

class Preparation_data():
    def __init__(self, type="label"):
        self.cols = ['k_id', 'k_text', 'k_box', 'v_id', 'v_text', 'v_box', 'k_embed', 'v_embed', 'width', 'height', 'fname']
        self.scaler_path = f"{root_re_model}/scaler.pkl"
        self.scaler = self.load_scaler()
        self.device = "cuda"
        self.kv_embed = KVEmbedding(self.device)
        self.type = type
    def preprocess_ser2re_batch_ver2(self, im_path, data = None):
            h, w = cv2.imread(im_path).shape[:2]
            f_name = os.path.basename(im_path)
            df_d = pd.DataFrame(data)
            # print(df_d)
            df_d['width'] = w
            df_d['height'] = h
            df_d['fname'] = f_name
            re = []
            # start_time = time.time()
            # print("--------======", df_d.transcription.values)
            if self.type == "label":
                df_key = df_d[df_d.label.str.lower()=='question']
                if len(df_key) == 0:
                    df_key = df_d[df_d.label.str.lower()=='key']
            else:
                df_key = df_d[df_d.pred.str.lower()=='key']
                if len(df_key) == 0:
                    df_key = df_d[df_d.pred.str.lower()=='question']
            if df_key.shape[0] == 0:
                # print("No question-answer pair was found")
                re = pd.DataFrame(re)
                return re
            df_key_transcription = df_key.transcription.values.tolist()
            df_key_transcription = self.kv_embed.embedding(df_key_transcription)
            df_key["embedding"] = df_key_transcription.tolist()
            
            if self.type == "label":
                df_value = df_d[df_d.label.str.lower()=='answer']
                if len(df_value) == 0:
                    df_value = df_d[df_d.label.str.lower()=='value']
            else:
                df_value = df_d[df_d.pred.str.lower()=='value']
                if len(df_value) == 0:
                    df_value = df_d[df_d.pred.str.lower()=='answer']
            if df_value.shape[0] == 0:
                # print("No question-answer pair was found")
                re = pd.DataFrame(re)
                return re
            df_value_transcription = df_value.transcription.values.tolist()
            df_value_transcription = self.kv_embed.embedding(df_value_transcription)
            df_value["embedding"] = df_value_transcription.tolist()
            
            for key in df_key.iterrows():
                linking = key[-1].linking
                for value in df_value.iterrows():
                    if [key[-1].id, value[-1].id] in linking:
                        link_label =1.0
                    else:
                        link_label =0.0
                    re.append({
                        'k_id': key[-1].id,
                        'k_text': str(key[-1].transcription),
                        'k_embed': key[-1].embedding,
                        'k_box': points2xyxy(key[-1].points),
                        'v_id': value[-1].id,
                        'v_text': value[-1].transcription,
                        'v_embed': value[-1].embedding,
                        'v_box': points2xyxy(value[-1].points),
                        'width': w,
                        'height': h,
                        'fname': os.path.basename(f_name),
                        'label': link_label
                    })
            # print("------------================time 2 for loops", time.time()-start_time)
            re = pd.DataFrame(re)
            if re.shape[0] == 0:
                # print("No question-answer pair was found")
                return re
        
            return re.reset_index(drop=True)

    def make_features(self, df:pd.DataFrame):
        """Create feature from dataframe

        Args:
            df (pd.DataFrame): input data

        Returns:
            pd.DataFrame: feature after process
        """
        # print(self.cols + ['label'])
        df = df[self.cols + ['label']]
        df.k_box = df.apply(lambda x: normalize_scale_bbox(x.k_box, x.width, x.height), axis=1)
        df.v_box = df.apply(lambda x:normalize_scale_bbox(x.v_box, x.width, x.height), axis=1)
        k_features = pd.DataFrame(df.k_box.tolist(), index=df.index, columns=['k_' + s for s in ['x1', 'y1', 'x2', 'y2']])
        v_features = pd.DataFrame(df.v_box.tolist(), index=df.index, columns=['v_' + s for s in ['x1', 'y1', 'x2', 'y2']])
        
        df = pd.concat([k_features, v_features, df[self.cols], df['label']], axis=1)
        
        df['k_cx'] = df.k_x1.add(df.k_x2).div(2)
        df['k_cy'] = df.k_y1.add(df.k_y2).div(2)
        
        df['v_cx'] = df.v_x1.add(df.v_x2).div(2)
        df['v_cy'] = df.v_y1.add(df.v_y2).div(2)
        
        df['fe1'] = abs(df.v_x1 - df.k_x1)
        df['fe2'] = abs(df.v_y1 - df.k_y1)
        df['fe3'] = abs(df.v_x1 - df.k_x2)
        df['fe4'] = abs(df.v_y1 - df.k_y2)
        df['fe5'] = abs(df.v_x2 - df.k_x1)
        df['fe6'] = abs(df.v_y2 - df.k_y1)
        df['fe7'] = abs(df.v_x2 - df.k_x2)
        df['fe8'] = abs(df.v_y2 - df.k_y2)
        df['fe9'] = abs(df.v_x2 - df.v_x1)
        df['fe10'] = abs(df.v_y2 - df.v_y1)
        df['fe11'] = abs(df.k_x2 - df.k_x1)
        df['fe12'] = abs(df.k_y2 - df.k_y1)
        
        df['fe13'] = df.apply(lambda x: cal_degrees([x.k_x1, x.k_y1], [x.v_x1, x.v_y1]), axis=1)
        df['fe14'] = df.apply(lambda x: cal_degrees([x.k_x2, x.k_y1], [x.v_x2, x.v_y1]), axis=1)
        df['fe15'] = df.apply(lambda x: cal_degrees([x.k_x2, x.k_y2], [x.v_x2, x.v_y2]), axis=1)
        df['fe16'] = df.apply(lambda x: cal_degrees([x.k_x1, x.k_y2], [x.v_x1, x.v_y2]), axis=1)
        df['fe17'] = df.apply(lambda x: cal_degrees([x['k_cx'], x['k_cy']], [x['v_cx'], x['v_cy']]), axis=1)
        
        df['fe18'] = df.apply(lambda x: boxes_distance([x.k_x1-x.v_x2, x.k_y2-x.v_y1],[x.v_x1-x.k_x2, x.v_y2-x.k_y1]), axis=1)
        df['fe19'] = df.apply(lambda x: dist_points([x.k_cx, x.k_cy], [x.v_cx, x.v_cy]), axis=1)
        
        k_embed_df = pd.DataFrame(np.array(df['k_embed'].values.tolist())).add_prefix('fe20')
        df = pd.concat([df, k_embed_df], axis=1)
        
        v_embed_df = pd.DataFrame(np.array(df['k_embed'].values.tolist())).add_prefix('fe21')
        df = pd.concat([df, v_embed_df], axis=1)
        
        cols = [c for c in df.columns if c.startswith('fe')] + ['label']

        return df[cols], df[self.cols]
    
    def load_scaler(self):
        # print('Loading scaler post processing relation ...')
        if os.path.exists(self.scaler_path):
            with open(self.scaler_path, 'rb') as f_scaler:
                scaler = pickle.load(f_scaler)
            f_scaler.close()
            return scaler 
        else:
            print("Path to scaler not exist !")
    
    def run(self, im_path, data):
        data = self.preprocess_ser2re_batch_ver2(im_path, data)
        if len(data)==0:
            return [], [], []
        d_features, __ = self.make_features(data)
        X, y = d_features.values[:, :-1], d_features.values[:, -1]
        X_transform = self.scaler.transform(X)
        return X_transform, y, data
# with open("/home/chuongphung/projects/chatgpt/XGBoost/dataset/20231030_re/train/train_original.json") as f:
#     data = json.load(f)


/home/chuongphung/.pyenv/versions/3.8.11/envs/information_extraction/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
def prepare_data_eval_ser(path_json_ser):
    path_image = os.path.dirname(path_json_ser)
    preparation_engine = Preparation_data("ser")
    
    X_val = np.array([])
    Y_val = np.array([])
    DF_val = []
    for doc_name in tqdm(os.listdir(path_json_ser)):
        if ".json" not in doc_name: continue
        with open(os.path.join(path_json_ser, doc_name)) as f:
            doc = json.load(f)
        im_path = os.path.join(path_image, doc_name.replace(".json", "_ser.jpg"))
        x_val, y_val, df_val = preparation_engine.run(im_path, doc)
        if len(y_val)!=0:
            if X_val.shape[0] == 0:
                X_val = x_val
            else:
                X_val = np.vstack((X_val, x_val))
            
            if Y_val.shape[0] == 0:
                Y_val = y_val
            else:
                Y_val = np.hstack((Y_val, y_val))
            if len(DF_val)==0:
                DF_val = df_val
            else:
                DF_val = pd.concat([DF_val, df_val], ignore_index=True,axis=0)
        # break
    print(X_val.shape)
    print(Y_val.shape)
    return X_val, Y_val, DF_val
    

In [10]:
def prepare_data_eval_label(path_json_ser):
    path_image = os.path.dirname(path_json_ser)
    
    preparation_engine = Preparation_data("label")
    
    X_val = np.array([])
    Y_val = np.array([])
    DF_val = []
    for doc_name in tqdm(os.listdir(path_json_ser)):
        if ".json" not in doc_name: continue
        with open(os.path.join(path_json_ser, doc_name)) as f:
            doc = json.load(f)
        im_path = os.path.join(path_image,doc_name.replace(".json", "_ser.jpg"))
        x_val, y_val, df_val = preparation_engine.run(im_path, doc)
        if len(y_val)!=0:
            if X_val.shape[0] == 0:
                X_val = x_val
            else:
                X_val = np.vstack((X_val, x_val))
            
            if Y_val.shape[0] == 0:
                Y_val = y_val
            else:
                Y_val = np.hstack((Y_val, y_val))
            if len(DF_val)==0:
                DF_val = df_val
            else:
                DF_val = pd.concat([DF_val, df_val], ignore_index=True,axis=0)
        # break
    print(X_val.shape)
    print(Y_val.shape)
    return X_val, Y_val, DF_val
    

In [11]:
def post_process(df: pd.DataFrame, threshold = 0.25):
    # one value only links to one key but one key can link to many value
    # df['pred_prob'] = pred_prob
    df['is_linking'] = 0
    fnames = df.fname.unique().tolist()
    for fname in fnames:
        df_fname = df[df.fname==fname]
        v_ids = df_fname.v_id.unique().tolist()
        for v_id in v_ids:
            df_vid = df_fname[df_fname.v_id==v_id]
            idx_max = df_vid.pred_prob.idxmax()

            if df.loc[(df.fname==fname)&(df.v_id==v_id)&(df.index==idx_max), 'pred_prob'].values[0] >= threshold:
                df.loc[(df.fname==fname)&(df.v_id==v_id)&(df.index==idx_max), 'is_linking'] = 1
    return df

In [12]:
def calculate_metric(DF_val_ser, Y_val_ser, DF_val_label, threadhold=0):
    new_df_1 = post_process(DF_val_ser.copy(), threadhold)
    tn, fp, fn, tp =  confusion_matrix(Y_val_ser, new_df_1["is_linking"]).ravel()
    print(tn, fp, fn, tp)
    number_positive = len(DF_val_label[DF_val_label["label"]==1])
    recall = tp/number_positive
    precision = tp/(tp+fp)
    f1 = 2/((recall+precision)/(precision*recall))
    print("recall", recall)
    print("precision", precision)
    print("f1", f1)

## dataset 1

In [13]:
X_val_ser1, Y_val_ser1, DF_val_ser1 = prepare_data_eval_ser(root_json_data1)
X_val_label1, Y_val_label1, DF_val_label1 = prepare_data_eval_label(root_json_data1)


100%|██████████| 135/135 [00:21<00:00,  6.19it/s]


(12905, 787)
(12905,)


100%|██████████| 135/135 [00:21<00:00,  6.27it/s]

(13258, 787)
(13258,)


In [14]:
pred_lightgbm = model_lightgbm.predict(X_val_ser1)
DF_val_ser1["pred_prob"] = model_lightgbm.predict_proba(X_val_ser1)[:, 1].tolist()


### Calculate metrics

In [15]:
# thread 0
calculate_metric(DF_val_ser1, Y_val_ser1, DF_val_label1, )

11435 210 26 1234
recall 0.8371777476255088
precision 0.8545706371191135
f1 0.8457847840986976


In [16]:
# thread 0.25
calculate_metric(DF_val_ser1, Y_val_ser1, DF_val_label1, 0.25)

11517 128 85 1175
recall 0.7971506105834464
precision 0.9017651573292402
f1 0.8462369463449766


## dataset 2

In [17]:
X_val_ser2, Y_val_ser2, DF_val_ser2 = prepare_data_eval_ser(root_json_data2)
X_val_label2, Y_val_label2, DF_val_label2 = prepare_data_eval_label(root_json_data2)


100%|██████████| 517/517 [01:13<00:00,  7.04it/s]


(51030, 787)
(51030,)


100%|██████████| 517/517 [01:24<00:00,  6.14it/s]

(67685, 787)
(67685,)


In [18]:
pred_lightgbm = model_lightgbm.predict(X_val_ser2)
DF_val_ser2["pred_prob"] = model_lightgbm.predict_proba(X_val_ser2)[:, 1].tolist()

### Calculate metrics

In [19]:
# thread 0
calculate_metric(DF_val_ser2, Y_val_ser2, DF_val_label2, 0)

46661 1352 176 2841
recall 0.6827685652487383
precision 0.6775578344860482
f1 0.6801532200143644


In [20]:
# thread 0.25
calculate_metric(DF_val_ser2, Y_val_ser2, DF_val_label2, 0.25)

47285 728 441 2576
recall 0.6190819514539774
precision 0.7796610169491526
f1 0.6901540522438043
